# TSPTW by OR-Tools

In [1]:
import random as rd
import numpy as np
import pandas as pd
import math
import copy as copy
from ortools.constraint_solver import routing_enums_pb2 
from ortools.constraint_solver import pywrapcp 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [131]:
def create_data_model():
    data = {}
    data["locations"] = [(round(np.random.uniform(0, MAX_X),2), round(np.random.uniform(0, MAX_Y),2)) for i in range(NUM_NODES)]
    data["locations"][0] = (MAX_X / 2, MAX_Y / 2)
    data["distance_matrix"] = compute_distance_matrix(data["locations"])
    data["time_matrix"] = compute_time_matrix(data["distance_matrix"])
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

def compute_distance_matrix(nodes):
    distance_matrix = []
    for node_i in nodes:
        distances = []
        for node_j in nodes:
            if node_i == node_j:
                distances.append(0)
            else:
                manhattan_distance = np.sum(np.abs(np.array(node_i) - np.array(node_j)))
                distances.append(round(manhattan_distance, 2))
        distance_matrix.append(distances)
    return distance_matrix

def compute_time_matrix(distance_matrix):
    time_matrix = []
    for i in distance_matrix:
        time_matrix.append(list(map(lambda x: int(x * 60), i)))
    return time_matrix

In [151]:
def main():

    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        0,  # allow waiting time
        640 * 60,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.time_limit.seconds = 10
    search_parameters.solution_limit = 100

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_second = print_solution(data, manager, routing, solution)
        cumul_data = get_cumul_data(solution, routing, time_dimension)
        return total_second
    else:
        print('No solution.')

def get_cumul_data(solution, routing, dimension):
    cumul_data = []
    for route_nbr in range(routing.vehicles()):
        route_data = []
        index = routing.Start(route_nbr)
        dim_var = dimension.CumulVar(index)
        route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            dim_var = dimension.CumulVar(index)
            route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
            cumul_data.append(route_data)
    return cumul_data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}s")
    print(f"Total serivce time: {(NUM_NODES - 1) * 60}s")
    return (total_time + (NUM_NODES - 1) * 60)
    

In [204]:
NUM_NODES = 300
MAX_X, MAX_Y = (20, 20)
data = create_data_model()
pd.DataFrame(data)

,locations,distance_matrix,time_matrix,num_vehicles,depot
0,"(10.0, 10.0)","[0, 4.67, 15.37, 10.66, 7.53, 10.21, 11.79, 13...","[0, 280, 922, 639, 451, 612, 707, 805, 597, 35...",1,0
1,"(10.45, 14.22)","[4.67, 0, 10.7, 6.89, 2.86, 13.98, 7.12, 8.76,...","[280, 0, 642, 413, 171, 838, 427, 525, 877, 89...",1,0
2,"(15.67, 19.7)","[15.37, 10.7, 0, 16.17, 7.84, 19.78, 3.58, 4.9...","[922, 642, 0, 970, 470, 1186, 214, 296, 1519, ...",1,0
3,"(4.27, 14.93)","[10.66, 6.89, 16.17, 0, 8.73, 20.87, 12.59, 14...","[639, 413, 970, 0, 523, 1252, 755, 853, 646, 5...",1,0
4,"(12.8, 14.73)","[7.53, 2.86, 7.84, 8.73, 0, 12.14, 4.26, 5.9, ...","[451, 171, 470, 523, 0, 728, 255, 354, 1049, 9...",1,0
...,...,...,...,...,...
295,"(13.97, 9.19)","[4.78, 8.55, 12.21, 15.44, 6.71, 7.57, 8.63, 1...","[286, 513, 732, 926, 402, 454, 517, 616, 787, ...",1,0
296,"(10.52, 0.9)","[9.62, 13.39, 23.95, 20.28, 16.11, 4.17, 20.37...","[577, 803, 1437, 1216, 966, 250, 1222, 1320, 5...",1,0
297,"(17.51, 9.61)","[7.9, 11.67, 11.93, 18.56, 9.83, 11.53, 10.29,...","[474, 700, 715, 1113, 589, 691, 617, 419, 1024...",1,0
298,"(3.56, 3.65)","[12.79, 17.46, 28.16, 11.99, 20.32, 10.3, 24.5...","[767, 1047, 1689, 719, 1219, 618, 1474, 1573, ...",1,0


In [202]:
# 38,400s
total_second = main()
total_second

Objective: 12563
Route for vehicle 0:
0 Time(0,0) -> 121 Time(24,24) -> 109 Time(36,36) -> 92 Time(45,45) -> 161 Time(84,84) -> 320 Time(104,104) -> 139 Time(112,112) -> 203 Time(151,151) -> 299 Time(182,182) -> 375 Time(234,234) -> 346 Time(239,239) -> 30 Time(262,262) -> 328 Time(296,296) -> 183 Time(337,337) -> 354 Time(344,344) -> 115 Time(372,372) -> 32 Time(404,404) -> 261 Time(430,430) -> 252 Time(491,491) -> 16 Time(536,536) -> 44 Time(560,560) -> 17 Time(620,620) -> 341 Time(729,729) -> 351 Time(812,812) -> 342 Time(891,891) -> 193 Time(943,943) -> 365 Time(962,962) -> 2 Time(1058,1058) -> 384 Time(1082,1082) -> 251 Time(1140,1140) -> 334 Time(1170,1170) -> 396 Time(1201,1201) -> 228 Time(1250,1250) -> 119 Time(1292,1292) -> 286 Time(1316,1316) -> 399 Time(1342,1342) -> 427 Time(1369,1369) -> 176 Time(1375,1375) -> 112 Time(1393,1393) -> 256 Time(1411,1411) -> 227 Time(1425,1425) -> 404 Time(1487,1487) -> 152 Time(1569,1569) -> 36 Time(1589,1589) -> 402 Time(1622,1622) -> 80 T

38303

In [203]:
result = []

for i in range(30):
    data = create_data_model()
    total_second = main()
    result.append(total_second)

np.mean(result)

Objective: 12205
Route for vehicle 0:
0 Time(0,0) -> 368 Time(61,61) -> 149 Time(131,131) -> 322 Time(158,158) -> 25 Time(210,210) -> 132 Time(246,246) -> 111 Time(255,255) -> 260 Time(270,270) -> 191 Time(288,288) -> 379 Time(315,315) -> 399 Time(325,325) -> 320 Time(335,335) -> 109 Time(372,372) -> 291 Time(387,387) -> 419 Time(415,415) -> 213 Time(432,432) -> 161 Time(440,440) -> 305 Time(461,461) -> 240 Time(488,488) -> 201 Time(592,592) -> 416 Time(612,612) -> 389 Time(628,628) -> 78 Time(659,659) -> 417 Time(683,683) -> 150 Time(717,717) -> 311 Time(729,729) -> 393 Time(745,745) -> 36 Time(765,765) -> 231 Time(780,780) -> 395 Time(803,803) -> 164 Time(829,829) -> 340 Time(848,848) -> 304 Time(878,878) -> 218 Time(894,894) -> 242 Time(909,909) -> 211 Time(949,949) -> 281 Time(988,988) -> 179 Time(1023,1023) -> 292 Time(1045,1045) -> 17 Time(1066,1066) -> 400 Time(1099,1099) -> 301 Time(1140,1140) -> 10 Time(1159,1159) -> 279 Time(1169,1169) -> 358 Time(1182,1182) -> 228 Time(1191,

38492.26666666667